In [3]:
import pandas	 as pd
import numpy	 as np
import matplotlib.pyplot as plt
import seaborn	 as sns
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import plotly.express as px
import plotly.figure_factory as ff
import plotly.subplots as sp

In [4]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv(r"C:\Users\vibhu\Downloads\MSc 4th semester\ATM cash relenishment model\project 100atm\merged_atm_data.csv")
df.head()



,ATM_ID,Bank_Name,Neighborhood,ATM_Type,Latitude,Longitude,Area_Type,Landmark,Date,Day,Time,Transaction_Type,Transaction_Amount,Remaining_Cash,Cash_Refill,Transaction_Status,Festival
0,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,Monday,00:05:23,Withdrawal,19300,1180700,NaN,Completed,NaN
1,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,Monday,00:06:23,Withdrawal,16000,1164700,NaN,Completed,NaN
2,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,Monday,00:09:24,Inquiry,0,1164700,NaN,Completed,NaN
3,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,Monday,00:14:34,Withdrawal,19000,1145700,NaN,Completed,NaN
4,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,Monday,00:18:03,Withdrawal,19700,1126000,NaN,Completed,NaN


In [5]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()
df['Week'] = df['Date'].dt.isocalendar().week
df['Quarter'] = df['Date'].dt.quarter
df['is_weekend'] = df['Weekday'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
df['is_weekday'] = df['Weekday'].apply(lambda x: 1 if x not in ['Saturday', 'Sunday'] else 0)
df['is_holiday'] = df['Weekday'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
df['is_holiday'] = df['is_holiday'].astype(int)

df.head()
df.drop(columns=['Year', 'Month', 'Week', 'Quarter'], inplace=True)
df.head()
# Convert categorical columns to category type
df['ATM_ID'] = df['ATM_ID'].astype('category')

In [6]:
# Create a new DataFrame with the count of transactions per day
transactions_per_day = df['Date'].dt.date.value_counts().sort_index().reset_index()
transactions_per_day.columns = ['Date', 'Number of Transactions']

# Plot the number of transactions per day
fig = px.bar(transactions_per_day, x='Date', y='Number of Transactions', 
			 labels={'Date': 'Date', 'Number of Transactions': 'Number of Transactions'}, 
			 title='Number of Transactions per Day')
fig.update_layout(xaxis_title='Date', yaxis_title='Number of Transactions', xaxis_tickangle=-45)
fig.show()




In [7]:
transactions_per_day = df.groupby('Date')['Transaction_Amount'].sum().reset_index()
transactions_per_day
# Plot the total withdrawal amount per day
fig = px.bar(transactions_per_day, x='Date', y='Transaction_Amount', 
			 labels={'Date': 'Date', 'Transaction_Amount': 'Total Withdrawal Amount'}, 
			 title='Total Withdrawal Amount per Day')
fig.update_layout(xaxis_title='Date', yaxis_title='Total Withdrawal Amount', xaxis_tickangle=-45)
fig.show()




c:\Users\vibhu\miniconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [8]:
# Identify salary dates (assuming salaries are credited on the 1st to 8 of every month)
df['is_salary_day'] = np.where(df['Date'].dt.day <= 8, 1, 0)
# Check if salary dates are identified correctly
print(df[['Date', 'is_salary_day']].head())


holiday_df= pd.read_csv(r'C:\Users\vibhu\Downloads\MSc 4th semester\ATM cash relenishment model\project 100atm\festivals.csv')
holiday_df.head()
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'], format='%Y-%m-%d', errors='coerce')
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'], format='%Y-%m-%d', errors='coerce')


# Ensure 'Date' column in holiday_df is in datetime format
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'], format='%Y-%m-%d', errors='coerce')

# Merge datasets on the date column
df = df.merge(holiday_df, how='left', left_on='Date', right_on='Date')  

# Fill missing values (for non-holiday days)
df['Festival Name'].fillna("No Holiday", inplace=True)

# Create a binary holiday feature
df['is_holiday'] = (df['Festival Name'] != "No Holiday").astype(int)

print(df[['Date', 'Festival Name', 'is_holiday']].head())


holiday_distribution = df['is_holiday'].value_counts()
print(holiday_distribution)

df.head()



        Date  is_salary_day
0 2024-01-01              1
1 2024-01-01              1
2 2024-01-01              1
3 2024-01-01              1
4 2024-01-01              1


C:\Users\vibhu\AppData\Local\Temp\ipykernel_13976\3907861838.py:20: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





        Date Festival Name  is_holiday
0 2024-01-01    No Holiday           0
1 2024-01-01    No Holiday           0
2 2024-01-01    No Holiday           0
3 2024-01-01    No Holiday           0
4 2024-01-01    No Holiday           0
is_holiday
0    8602906
1     918413
Name: count, dtype: int64


,ATM_ID,Bank_Name,Neighborhood,ATM_Type,Latitude,Longitude,Area_Type,Landmark,Date,Day,...,Remaining_Cash,Cash_Refill,Transaction_Status,Festival,Weekday,is_weekend,is_weekday,is_holiday,is_salary_day,Festival Name
0,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,1,...,1180700,NaN,Completed,NaN,Monday,0,1,0,1,No Holiday
1,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,1,...,1164700,NaN,Completed,NaN,Monday,0,1,0,1,No Holiday
2,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,1,...,1164700,NaN,Completed,NaN,Monday,0,1,0,1,No Holiday
3,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,1,...,1145700,NaN,Completed,NaN,Monday,0,1,0,1,No Holiday
4,ATM_001,Bank of Baroda,Indira Nagar,Off-site,26.698003,80.848009,Transport Hub,Railway Station,2024-01-01,1,...,1126000,NaN,Completed,NaN,Monday,0,1,0,1,No Holiday


In [9]:
# 3. Monthly Transaction Trends

# Ensure the 'Month' column exists
df['Month'] = df['Date'].dt.month

# Calculate monthly transaction trends
monthly_trends = df.groupby('Month')['Transaction_Amount'].sum()

# Plot the monthly transaction trends
monthly_trends_fig = px.bar(
	monthly_trends,
	x=monthly_trends.index,
	y=monthly_trends.values,
	labels={"x": "Month", "y": "Total Transaction Amount"},
	title="Monthly Transaction Trends"
)
monthly_trends_fig.update_xaxes(
	tickvals=list(range(1, 13)),
	ticktext=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
)
monthly_trends_fig.show()

In [10]:
# Group the data by 'ATM_Type'
atm_type_clusters = df.groupby('ATM_Type')

# Analyze data within each ATM type cluster
for atm_type, cluster_data in atm_type_clusters:
	print(f"Cluster: {atm_type}")
	print(f"Number of ATMs: {cluster_data['ATM_ID'].nunique()}")
	print(f"Total Transactions: {cluster_data['Transaction_Amount'].sum()}")
	print(f"Average Transaction Amount: {cluster_data['Transaction_Amount'].mean()}")
	print(f"Remaining Cash (Average): {cluster_data['Remaining_Cash'].mean()}")
	print(f"Number of Holidays: {cluster_data['is_holiday'].sum()}")
	print(f"Number of Salary Days: {cluster_data['is_salary_day'].sum()}")
	print("-" * 50)


Cluster: Brown Label
Number of ATMs: 9
Total Transactions: 9046762900
Average Transaction Amount: 13467.27457317282
Remaining Cash (Average): 120287.3932466852
Number of Holidays: 66258
Number of Salary Days: 226291
--------------------------------------------------
Cluster: Off-site
Number of ATMs: 50
Total Transactions: 62532602200
Average Transaction Amount: 13488.519747790491
Remaining Cash (Average): 103127.43452473011
Number of Holidays: 444875
Number of Salary Days: 1557229
--------------------------------------------------
Cluster: On-site
Number of ATMs: 21
Total Transactions: 35388200600
Average Transaction Amount: 13201.96625661018
Remaining Cash (Average): 57814.19158560356
Number of Holidays: 256930
Number of Salary Days: 899499
--------------------------------------------------
Cluster: White Label
Number of ATMs: 20
Total Transactions: 20650576400
Average Transaction Amount: 13470.273859657362
Remaining Cash (Average): 120271.29665868258
Number of Holidays: 150350
Number

In [11]:
# adf test for stationarity
from statsmodels.tsa.stattools import adfuller
def adf_test(series):
	result = adfuller(series)
	print('ADF Statistic:', result[0])
	print('p-value:', result[1])
	print('Critical Values:')
	for key, value in result[4].items():
		print(f'   {key}: {value}')
	# Interpret the result
	if result[1] <= 0.05:
		print("Reject the null hypothesis (series is stationary)")
	else:
		print("Fail to reject the null hypothesis (series is non-stationary)")	

# Define the time_series_data DataFrame
time_series_data = df[['Date', 'Transaction_Amount']].copy()
time_series_data.set_index('Date', inplace=True)



# Perform ADF test on the time series data
# Perform ADF test on the aggregated daily transaction amounts
daily_transaction_amounts = transactions_per_day['Transaction_Amount']
adf_test(daily_transaction_amounts)

# diifferencing
daily_transaction_amounts_diff = daily_transaction_amounts.diff().dropna()
adf_test(daily_transaction_amounts_diff)

ADF Statistic: -2.690243158951738
p-value: 0.07574189977895643
Critical Values:
   1%: -3.449226932880019
   5%: -2.869857365438656
   10%: -2.571201085130664
Fail to reject the null hypothesis (series is non-stationary)
ADF Statistic: -5.954805284079538
p-value: 2.1055603949379332e-07
Critical Values:
   1%: -3.449226932880019
   5%: -2.869857365438656
   10%: -2.571201085130664
Reject the null hypothesis (series is stationary)


In [12]:
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import plotly.graph_objects as go

# Group the data by ATM_Type
clustered_data = df.groupby('ATM_Type')

# Iterate through each ATM type cluster and fit models
for atm_type, cluster_data in clustered_data:
	print(f"Processing ATM Type: {atm_type}")
	
	# Ensure the 'Transaction_Amount' column is used for modeling
	time_series = cluster_data[['Date', 'Transaction_Amount', 'is_salary_day', 'is_holiday']].copy()
	time_series.set_index('Date', inplace=True)
	time_series = time_series.resample('D').sum()
	
	# Define the exogenous variables
	exog_variables = time_series[['is_salary_day', 'is_holiday']]
	
	try:
		# Fit auto ARIMA model with exogenous variables
		Auto_arima_model = auto_arima(
			time_series['Transaction_Amount'], 
			exogenous=exog_variables, 
			seasonal=True, 
			m=7
		)
		
		# Fit SARIMAX Model with exogenous variables
		Sarimax_model = SARIMAX(
			time_series['Transaction_Amount'], 
			order=(1, 1, 1), 
			seasonal_order=(1, 1, 1, 7), 
			exog=exog_variables
		)
		Sarimax_model_fit = Sarimax_model.fit()
		
		# Print model summaries
		print(f"Auto ARIMA Model Summary for ATM Type {atm_type}:")
		print(Auto_arima_model.summary())
		print(f"\nSARIMAX Model Summary for ATM Type {atm_type}:")
		print(Sarimax_model_fit.summary())
		
		# Calculate fitted values
		fitted_values = Auto_arima_model.predict_in_sample()
		
		# Calculate RMSE, MAE, and MAPE
		rmse = sqrt(mean_squared_error(time_series['Transaction_Amount'], fitted_values))
		mae = mean_absolute_error(time_series['Transaction_Amount'], fitted_values)
		mape = np.mean(np.abs((time_series['Transaction_Amount'] - fitted_values) / time_series['Transaction_Amount'])) * 100
		print(f"ATM Type {atm_type} - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}%")
		
		# Plot the actual vs fitted values
		fig = go.Figure()
		fig.add_trace(go.Scatter(
			x=time_series.index,
			y=time_series['Transaction_Amount'],
			mode='lines',
			name='Actual'
		))
		fig.add_trace(go.Scatter(
			x=time_series.index,
			y=fitted_values,
			mode='lines',
			name='Fitted'
		))
		fig.update_layout(
			title=f"ATM Type {atm_type} - Actual vs Fitted Values",
			xaxis_title="Date",
			yaxis_title="Transaction Amount",
			template="plotly_white"
		)
		fig.show()
		
	except Exception as e:
		print(f"Error processing ATM Type {atm_type}: {e}")


Processing ATM Type: Brown Label


c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: Fu

Auto ARIMA Model Summary for ATM Type Brown Label:
                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                  366
Model:             SARIMAX(4, 0, 3)x(1, 0, [1], 7)   Log Likelihood               -6314.076
Date:                             Sun, 20 Apr 2025   AIC                          12650.152
Time:                                     01:06:25   BIC                          12693.081
Sample:                                 01-01-2024   HQIC                         12667.211
                                      - 12-31-2024                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   3.731e+06   4.84e-08   7.71e+13      0.000 

Processing ATM Type: Off-site


c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: Fu

...

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: Fu

Auto ARIMA Model Summary for ATM Type On-site:
                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                  366
Model:             SARIMAX(2, 0, 3)x(1, 0, [1], 7)   Log Likelihood               -6831.758
Date:                             Sun, 20 Apr 2025   AIC                          13681.516
Time:                                     01:08:06   BIC                          13716.639
Sample:                                 01-01-2024   HQIC                         13695.473
                                      - 12-31-2024                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   1.469e+07   1.09e-08   1.35e+15      0.000    1

Processing ATM Type: White Label


c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: Fu

Auto ARIMA Model Summary for ATM Type White Label:
                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                  366
Model:             SARIMAX(2, 0, 3)x(1, 0, [1], 7)   Log Likelihood               -6597.714
Date:                             Sun, 20 Apr 2025   AIC                          13213.427
Time:                                     01:09:52   BIC                          13248.551
Sample:                                 01-01-2024   HQIC                         13227.385
                                      - 12-31-2024                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   8.949e+06    1.3e-08    6.9e+14      0.000 

In [13]:
# Group data by ATM_Type
atm_type_forecasts = {}

for atm_type, cluster_data in df.groupby('ATM_Type'):
	# Resample the time series data for the current ATM type
	cluster_time_series = cluster_data[['Date', 'Transaction_Amount', 'is_salary_day', 'is_holiday']].copy()
	cluster_time_series.set_index('Date', inplace=True)
	cluster_time_series = cluster_time_series.resample('D').sum()

	# Prepare exogenous variables for January 2025 for the current ATM type
	january_2025_exog = pd.DataFrame({
		'is_salary_day': [1 if day <= 8 else 0 for day in range(1, 32)],
		'is_holiday': [0] * 31  # Assuming no holidays in January 2025; update if needed
	})

	# Predict January 2025 for the current ATM type using both models
	january_2025_forecast = pd.DataFrame({
		'Date': pd.date_range(start='2025-01-01', end='2025-01-31', freq='D'),
		'Auto_ARIMA_Forecast': Auto_arima_model.predict(n_periods=31),
		'SARIMAX_Forecast': Sarimax_model_fit.get_forecast(steps=31, exog=january_2025_exog).predicted_mean
	})

	# Store the forecast for the current ATM type
	atm_type_forecasts[atm_type] = january_2025_forecast

	# Plot the predictions for the current ATM type
	fig = go.Figure()

	# Add Auto ARIMA predictions
	fig.add_trace(go.Scatter(
		x=january_2025_forecast['Date'],
		y=january_2025_forecast['Auto_ARIMA_Forecast'],
		mode='lines+markers',
		name=f'Auto ARIMA Predictions ({atm_type})',
		line=dict(color='blue')
	))

	# Add SARIMAX predictions
	fig.add_trace(go.Scatter(
		x=january_2025_forecast['Date'],
		y=january_2025_forecast['SARIMAX_Forecast'],
		mode='lines+markers',
		name=f'SARIMAX Predictions ({atm_type})',
		line=dict(color='red')
	))

	# Customize the layout
	fig.update_layout(
		title=f'January 2025 Predictions: {atm_type}',
		xaxis_title='Date',
		yaxis_title='Transaction Amount',
		legend_title='Model',
		template='plotly_white'
	)

	# Show the graph
	fig.show()

# The `atm_type_forecasts` dictionary contains predictions for each ATM type.


c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\vibhu\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\vibhu\miniconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [14]:
# Filter data for on-site ATMs
on_site_data = df[df['ATM_Type'] == 'On-Site']

# Resample the time series data for on-site ATMs
on_site_time_series = on_site_data[['Date', 'Deposit_Amount', 'is_salary_day', 'is_holiday']].copy()
on_site_time_series.set_index('Date', inplace=True)
on_site_time_series = on_site_time_series.resample('D').sum()

# Prepare exogenous variables for January 2025
january_2025_exog = pd.DataFrame({
	'is_salary_day': [1 if day <= 8 else 0 for day in range(1, 32)],
	'is_holiday': [0] * 31  # Assuming no holidays in January 2025; update if needed
})

# Predict January 2025 deposit amounts for on-site ATMs
january_2025_forecast = pd.DataFrame({
	'Date': pd.date_range(start='2025-01-01', end='2025-01-31', freq='D'),
	'Auto_ARIMA_Forecast': Auto_arima_model.predict(n_periods=31),
	'SARIMAX_Forecast': Sarimax_model_fit.get_forecast(steps=31, exog=january_2025_exog).predicted_mean
})

# Plot the predictions for on-site ATMs
fig = go.Figure()

# Add Auto ARIMA predictions
fig.add_trace(go.Scatter(
	x=january_2025_forecast['Date'],
	y=january_2025_forecast['Auto_ARIMA_Forecast'],
	mode='lines+markers',
	name='Auto ARIMA Predictions (On-Site)',
	line=dict(color='blue')
))

# Add SARIMAX predictions
fig.add_trace(go.Scatter(
	x=january_2025_forecast['Date'],
	y=january_2025_forecast['SARIMAX_Forecast'],
	mode='lines+markers',
	name='SARIMAX Predictions (On-Site)',
	line=dict(color='red')
))

# Customize the layout
fig.update_layout(
	title='January 2025 Deposit Predictions: On-Site ATMs',
	xaxis_title='Date',
	yaxis_title='Deposit Amount',
	legend_title='Model',
	template='plotly_white'
)

# Show the graph
fig.show()

KeyError: "['Deposit_Amount'] not in index"